## Web scraping do site do TRF5

### A partir do relatório enviado pelo TRF5 com os precatórios que tiveram valores efetivamente depositados em conta judicial, esse scraping vai ao site do TRF5 e obtém os dados de data de depósito efetiva, instituição financeira (Caixa ou BB)

### De posse desses dados, é possível diligenciar à instituição financeira para se obter o extrato da conta judicial relacionada para se saber: 1) se o valor depositado na conta judicial foi levantado pelo município 2) se o advogado levantou o valor de seus honorários 3) para qual conta o município transferiu o valor


In [38]:
import requests
from bs4 import BeautifulSoup

In [39]:
parametros = {'filtro': 'PRC103869', 'tipo': 'xmlproc', 'ordenacao cpf': 'D', 'tipoproc': 'T'}
try
    r = requests.post('http://www4.trf5.jus.br/cp/cp.do', data=parametros)
except HTTPError as e:
    print(e)
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="pt">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Tribunal Regional Federal da 5ª Região - Resultado Consulta Processual
  </title>
  <base href="http://www4.trf5.jus.br/cp/cp.do"/>
  <link href="http://www.trf5.jus.br/modules/opensearchTRF5_porNum.xml" rel="search" title="TRF5 - Consulta Processual por Nº do processo" type="application/opensearchdescription+xml"/>
  <link href="http://www.trf5.jus.br/modules/opensearchTRF5_porNome.xml" rel="search" title="TRF5 - Consulta Processual por Nome de Parte" type="application/opensearchdescription+xml"/>
  <link href="http://www.trf5.jus.br/modules/opensearchTRF5_porNumOrig.xml" rel="search" title="TRF5 - Consulta Processual por Nº do originário" type="application/opensearchdescription+xml"/>
  <link href="http://www.trf5.jus.br/modules/opensearchTRF5_porOAB.xml" r

### Lendo o objeto soup para pegar seus componentes

In [45]:
# pega os macro objetos
[type(i) for i in list(soup.children)]

[bs4.element.NavigableString,
 bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [46]:
html_trf = list(soup.children)[3]

In [129]:
def verifica_tabela(tabela):
    flag_requerente = False
    flag_advogado = False
    # percorre as linhas da tabela
    celulas =  tabela.find_all('td')
    for celula in celulas:
        # se achamos a linha com o nome do requerente...
        if 'REQTE' in celula.text.upper():
            flag_requerente = True
            continue
            
        if 'advogado' in celula.text.lower():
            flag_advogado = True
            continue  
        
        # se estamos na célula do requerente..    
        if flag_requerente:    
            # desliga a flag para não entrarmos mais aqui
            flag_requerente = False
            req_nome = celula.text.strip().replace(':', '').strip()            
            print(req_nome)            
        
        if flag_advogado:
            flag_advogado = False
            advog_nome = celula.text.strip().replace(':', '').strip()
            print(advog_nome)            
            
def verifica_tabela_depositos(tabela):
    i = 0
    celulas = tabela.find_all('td')
    for celula in celulas:        
        # pega as informações de depósito    
          if ('depósito em') in celula.text.lower():
            print(celula.text, i, celulas[i-2].text.strip())       
          i += 1           

# pega todas as tabelas do html
tabelas = html_trf.find_all('table')
# percorre todas as tabelas em busca do que nos interessa
for tabela in tabelas:
    verifica_tabela(tabela) 
    verifica_tabela_depositos(tabela)
    

MUNICÍPIO DE ITABAIANA
FÁBIO ROMERO DE CARVALHO - PB011667
GUSTAVO BRAGA LOPES - PB012692
GUSTAVO BRAGA LOPES - PB012692
Depósito em Conta . 2 Em 02/10/2015 10:51
Depósito em Conta . 2 Em 04/11/2014 12:46


In [57]:
for sibling in html_trf.body.find(text='REQTE').next_siblings:
    print(sibling)